In [ ]:
import os
print(sorted(os.listdir()))

In [ ]:
import json


# file_path = "C:/Users/Administrator/Desktop/SFMAP/fintech_LLM/v2/06_finetuning/instruction-data.json"
file_path = "/kaggle/input/instructions/instruction-data.json"

with open(file_path, "r") as file:
    data = json.load(file)
print("Number of entries:", len(data))

In [ ]:
print("Example entry:\n", data[50])

In [ ]:
print("Another example entry:\n", data[199])

In [ ]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

In [ ]:
model_input = format_input(data[50])
desired_response = f"\n\n### Response:\n{data[50]['output']}"

print(model_input + desired_response)

In [ ]:
model_input = format_input(data[99])
desired_response = f"\n\n### Response:\n{data[199]['output']}"

print(model_input + desired_response)

In [ ]:
import json


file_path = "/kaggle/input/instructions/instruction-data.json"

with open(file_path, "r") as file:
    data = json.load(file)
print("Number of entries:", len(data))

In [ ]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.15)    # 15% for testing

train_data = data[:train_portion]
test_data = data[train_portion:]

In [ ]:
print("Training set length:", len(train_data))
print("Test set length:", len(test_data))

In [ ]:
with open("train.json", "w") as json_file:
    json.dump(train_data, json_file, indent=4)

with open("test.json", "w") as json_file:
    json.dump(test_data, json_file, indent=4)

In [ ]:
%pip install litgpt tqdm

In [ ]:
!litgpt finetune_lora microsoft/phi-2 \
  --data JSON \
  --data.val_split_fraction 0.1 \
  --data.json_path "train.json" \
  --train.epochs 3 \
  --train.log_interval 100 \
  --train.global_batch_size 1 \
  --train.micro_batch_size 1 \
  --train.lr_warmup_steps 1000 \
  --train.max_seq_length 512 \
  --precision bf16 \
  --devices 1 \
  --out_dir "output" \
  --logger_name "csv"

In [ ]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

print(format_input(test_data[0]))

In [ ]:
from litgpt import LLM

llm = LLM.load("microsoft/phi-2")

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(test_data))):
    response = llm.generate(test_data[i])
    test_data[i]["base_model"] = response

In [ ]:
test_data[1]

In [ ]:
import shutil

# Define the folder path and output zip name
output_folder = "/kaggle/working"
zip_file = "/kaggle/working/output.zip"

# Compress the folder
shutil.make_archive(base_name=zip_file.replace(".zip", ""), format="zip", root_dir=output_folder)

print("Folder compressed to:", zip_file)


In [ ]:
from litgpt import LLM

del llm
llm2 = LLM.load("/kaggle/working/out/finetune/lora/final/", device="cpu")

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(test_data))):
    response = llm2.generate(test_data[i])
    test_data[i]["finetuned_model"] = response

In [ ]:
!litgpt evaluate out/finetune/lora/final --tasks "mmlu_philosophy" --batch_size 4

In [ ]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

In [ ]:
import json

json_file = "test_response_before_after.json"

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

In [ ]:
json_data[0]

In [ ]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

print(format_input(json_data[0])) # input

In [ ]:
json_data[0]["output"]

In [ ]:
json_data[0]["response_before"]

In [ ]:
from litgpt import LLM

llm = LLM.load("meta-llama/Meta-Llama-3-8B-Instruct")

In [ ]:
from tqdm import tqdm


def generate_model_scores(json_data, json_key):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = llm.generate(prompt, max_new_tokens=50)
        try:
            scores.append(int(score))
        except ValueError:
            continue

    return scores

In [ ]:
for model in ("response_before", "response_after"):

    scores = generate_model_scores(json_data, model)
    print(f"\n{model}")
    print(f"Number of scores: {len(scores)} of {len(json_data)}")
    print(f"Average score: {sum(scores)/len(scores):.2f}\n")